In [1]:
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import os
import time
import pandas as pd
import io
import csv

In [2]:
## Loading protein-coding-genes data

df = pd.read_csv('/Users/jiamingxu/Library/CloudStorage/OneDrive-NortheasternUniversity/Fall 2022/STAT/github/XNProject/Dataset/protein-coding-genes.tsv', sep = '\t')
human_genes = df[df.GLGB_HUMAN.str.contains("HUMAN")]['GBE1'].unique()
#df = pd.read_csv('/Users/jiamingxu/Downloads/Achilles_gene_effect.csv', index_col = 0)

In [3]:
len(human_genes)

18898

In [4]:
gene_symbol = np.array(re.findall(' (\S+) ', " "+" ".join(df.columns)))
gene_id = np.array(re.findall('\((\S+)\)', " "+" ".join(df.columns)))

In [ ]:
## Human protein genes dataframe:
## gp_df
## gp_df_clean: filter genes only contain in CRISPR gene effect central nerverous system file

gp_df = pd.read_csv("/Users/jiamingxu/Library/CloudStorage/OneDrive-NortheasternUniversity/Fall 2022/STAT/github/XNProject/Dataset/protein-coding-genes.tsv"\
                    , sep='\t', names=['Description', 'Symbol', 'Protein_Name']).dropna().drop_duplicates('Symbol')
gp_df_clean = gp_df[gp_df['Symbol'].isin(gene_names)]
gp_df_clean = gp_df_clean[gp_df_clean['Protein_Name'].str.contains("HUMAN")]
#gp_df['Protein_Name'].str.contains("HUMAN")
gp_df_clean.reset_index()
protein_gene_interaction_names = gp_df_clean['Symbol']

In [4]:
## url for finding possible interactable genes on BIOGRID.

url = 'https://webservice.thebiogrid.org/interactions/?searchNames=true&geneList={}&searchbiogridids=true&includeInteractors=true&accessKey=f2451e613b27c3d6bee6e2dd5618dc2c'

In [1]:
json_url = 'https://webservice.thebiogrid.org/interactions/?searchNames=true&format=json&geneList=ABCC1&searchbiogridids=true&includeInteractors=true&includeHeader=true&accessKey=f2451e613b27c3d6bee6e2dd5618dc2c'

In [5]:
urlData = requests.get(json_url,timeout=2).content

In [13]:
pd.read_json(urlData.decode())

,594211,594329,747641,747977,748071,748631,748888,748997,749776,749982,...,3329819,3330206,3330731,3332856,3334493,3335825,3336374,3336513,3347692,3405902
BIOGRID_INTERACTION_ID,594211,594329,747641,747977,748071,748631,748888,748997,749776,749982,...,3329819,3330206,3330731,3332856,3334493,3335825,3336374,3336513,3347692,3405902
ENTREZ_GENE_A,340061,81622,8663,4363,4363,4363,4363,4363,4363,4363,...,58480,399,390,5880,389,171177,27289,8153,214,72512
ENTREZ_GENE_B,4363,4363,4363,25915,22827,63875,4715,6201,7419,51075,...,4363,4363,4363,4363,4363,4363,4363,4363,4363,17250
BIOGRID_ID_A,130988,123557,114212,110503,110503,110503,110503,110503,110503,110503,...,121812,106892,106883,111818,106882,128110,118113,113810,106716,215410
BIOGRID_ID_B,110503,110503,110503,117419,116502,121968,110795,112115,113262,119266,...,110503,110503,110503,110503,110503,110503,110503,110503,110503,201374
SYSTEMATIC_NAME_A,-,-,-,-,-,-,-,-,-,-,...,SB128,-,-,-,RP11-426L16.4,-,-,-,-,-
SYSTEMATIC_NAME_B,-,-,-,-,-,-,-,-,-,CGI-31,...,-,-,-,-,-,-,-,-,-,-
OFFICIAL_SYMBOL_A,TMEM173,UNC93B1,EIF3C,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,...,RHOU,RHOH,RND3,RAC2,RHOC,RHOV,RND1,RND2,ALCAM,Tmem173
OFFICIAL_SYMBOL_B,ABCC1,ABCC1,ABCC1,NDUFAF3,PUF60,MRPL17,NDUFB9,RPS7,VDAC3,TMX2,...,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,ABCC1,Abcc1
SYNONYMS_A,ERIS|MITA|MPYS|NET23|SAVI|STING|hMITA|hSTING,IIAE1|UNC93|UNC93B|Unc-93B1,EIF3CL|EIF3S8|eIF3-p110,ABC29|ABCC|GS-X|MRP|MRP1,ABC29|ABCC|GS-X|MRP|MRP1,ABC29|ABCC|GS-X|MRP|MRP1,ABC29|ABCC|GS-X|MRP|MRP1,ABC29|ABCC|GS-X|MRP|MRP1,ABC29|ABCC|GS-X|MRP|MRP1,ABC29|ABCC|GS-X|MRP|MRP1,...,ARHU|CDC42L1|G28K|WRCH1|hG28K,ARHH|TTF,ARHE|Rho8|RhoE|memB,EN-7|Gx|HSPC022|p21-Rac2,ARH9|ARHC|H9|RHOH9,ARHV|CHP|WRCH2,ARHS|RHO6|RHOS,ARHN|RHO7|RhoN,CD166|MEMD,2610307O08Rik|ERIS|MPYS|Mita|STING


In [9]:
rawData

,0
0,"{""594211"":{""BIOGRID_INTERACTION_ID"":594211,""EN..."


In [ ]:
# Getting edges using BioGRID REST API
n=0
nodes_edges = {}
for id_ in human_genes:
    while True:
        try:
            urlData = requests.get(url.format(id_), timeout=2).content
            break
        except:
            print("Waiting for refresh allowance.")
            time.sleep(30)
            continue
        
    if urlData != b"":
        rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')), sep='\t', header=None)
        a_ = list(np.unique(rawData.iloc[:, [7, 8]]))
        a_ = [ x.upper() for x in a_ ]
        a_.remove(id_.upper())
        nodes_edges[id_] = a_

    if n % 10 == 0:
        print("processing %i"%n, end='\r')
    n += 1

In [5]:
os.getcwd()

'/Users/jiamingxu/Library/CloudStorage/OneDrive-NortheasternUniversity/Fall 2022/STAT'

In [5]:
n = 0
csv_file = "Gene_interactions.csv"
csv_columns = ['Gene_','Connected_genes_']
with open(csv_file, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_columns)
    for key, value in nodes_edges.items():
        n += 1
        if key != "" and value != []:
            writer.writerow([key, value])
        else:
            print(key)
            

NameError: name 'nodes_edges' is not defined

In [222]:
new_dict = {key: val for key,
            val in nodes_edges.items() if len(val) != 1}

In [14]:
df = pd.read_csv('/Users/jiamingxu/Downloads/all_gene_interactions_n10.csv', index_col=1).drop(columns=['id'])

In [15]:
cancer_ty

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_22,x_23,x_24,x_25,x_26,x_27,x_28,x_29,x_30,x_31
node,,,,,,,,,,,,,,,,,,,,,
0610007P14RIK,-0.241660,0.438166,-0.013287,-0.055989,-0.241500,-0.601827,0.085589,0.506501,0.219237,-0.014464,...,0.470173,-0.083119,-0.046583,-0.182563,-0.640191,-0.408186,0.242811,-0.275450,-0.170636,-0.206503
0610010K14RIK,-0.120984,-0.095230,0.174372,-0.359242,-0.116310,-0.400138,0.431912,0.514591,0.490621,0.087008,...,0.065939,-0.368291,0.063295,0.052304,-0.774571,-0.824540,-0.182099,-0.401011,-0.350947,-0.548576
1110001J03RIK,-0.275524,0.311952,-0.191020,0.084877,-0.393019,-0.643902,0.216613,0.445031,0.310006,0.052862,...,0.311157,-0.282494,-0.034166,0.031568,-0.398391,-0.268042,0.123453,-0.230327,-0.124510,-0.433168
1110004E09RIK,-0.220656,0.122091,-0.006122,0.296347,-0.015953,-0.411766,-0.046867,0.143783,0.447478,0.186524,...,0.079783,-0.018830,0.191364,0.282159,-0.294113,-0.434377,-0.031974,-0.097271,0.107300,-0.022005
1110004F10RIK,-0.186386,-0.189858,-0.024390,0.358786,-0.288283,-0.904854,0.675721,-0.156558,0.511148,0.151499,...,-0.441885,-0.349014,0.774430,-0.305200,-0.156068,-1.091663,-0.095774,0.330863,-0.592771,-0.627109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,-0.826074,0.162869,0.463025,0.184096,-0.323719,-0.462886,-0.309116,0.407305,0.567634,0.298988,...,-0.009325,-0.161164,0.447260,-0.995392,-0.549543,-0.283312,0.026692,-0.219857,0.471839,-0.928996
ZYX,-0.337855,0.204556,0.687981,0.020556,0.302415,0.150949,0.177997,-0.207455,0.982822,0.693328,...,-0.082132,-0.156327,-0.224571,0.923025,-0.411934,-0.713231,0.041977,-0.032459,-0.146239,-0.211201
ZZEF1,-0.746778,0.525200,-0.545656,-0.198598,-0.210923,0.109058,0.525003,0.130418,0.465631,-0.131611,...,0.216444,-0.110389,0.460101,-1.117527,-1.095618,-0.743194,0.018654,-0.296020,0.278738,-0.589885


In [224]:
gene_interactions_df = pd.DataFrame(columns=['g1', 'g2'])
g1 = []
g2 = []

In [225]:
for key, values in new_dict.items():
    for v in values:
        g1.append(key)
        g2.append(v)

In [226]:
pd.DataFrame( [pd.Series(g1), pd.Series(g2)], columns = ['g1', 'g2'] )

,g1,g2
0,NaN,NaN
1,NaN,NaN


In [227]:
gene_interactions_df = pd.concat([pd.Series(g1), pd.Series(g2)], axis=1)
gene_interactions_df.columns = ['g1', 'g2']

In [34]:
gene_interactions_df.to_csv("Gene_interactions_in_pairs.csv")

NameError: name 'gene_interactions_df' is not defined

In [229]:
gene_interactions_df

,g1,g2
0,AGPAT1,A2M
1,AGPAT1,A2ML1
2,AGPAT1,AHNAK2
3,AGPAT1,AIM1
4,AGPAT1,AKR1B10
...,...,...
1842659,UBE2NL,MEPCE
1842660,UBE2NL,PHGDH
1842661,UBE2NL,PLEKHA4
1842662,VN1R5,CEP164


In [186]:
training_path = '/Users/jiamingxu/Library/CloudStorage/OneDrive-NortheasternUniversity/Fall 2022/STAT/github/XNProject/Dataset/training_set/'
training_csv_files = []
for file_ in os.scandir(training_path):
    if file_.name.endswith('csv'):
        training_csv_files.append(file_.name)

In [190]:
for file_ in training_csv_files:
    df_ = pd.read_csv(training_path + file_)
    gene_interactions_df[gene_interactions_df.loc[:, 'g1'].isin(df_.loc[:, 'gene'])].to_csv(file_.split('_')[0] + "_gene_connections.csv")

In [177]:
df_traning_pancreatic_df = pd.read_csv("/Users/jiamingxu/Library/CloudStorage/OneDrive-NortheasternUniversity/Fall 2022/STAT/github/XNProject/Dataset/training_set/pancreatic_training_genes_set.csv")

In [181]:
gene_interactions_df[gene_interactions_df.loc[:, 'g1'].isin(df_traning_pancreatic_df.loc[:, 'gene'])].to_csv('')

,g1,g2
6580,PFKFB2,BMH1
6581,PFKFB2,BMH2
6582,PFKFB2,CBY1
6583,PFKFB2,COPS2
6584,PFKFB2,CPSF7
...,...,...
1840236,ZNF99,KRAS
1841430,ZPBP,CALU
1841431,ZPBP,DPPA3
1841432,ZPBP,ERRFI1


In [191]:
human_genes

array(['AGPAT1', 'AGPAT2', 'AGPAT3', ..., 'XKRY', 'XKRY2', 'ZNF788P'],
      dtype=object)

In [278]:
n = 0
sym_2_id = {}
for g in list(set(gene_interactions_df_clean.g1)) + list(set(gene_interactions_df_clean.g2)):
    sym_2_id[g] = n
    n += 1

In [279]:
id_2_sym = {}

for k, v in sym_2_id.items():
    id_2_sym[v] = k

In [6]:
gene_interactions_df_clean

NameError: name 'gene_interactions_df_clean' is not defined

In [281]:
sym_2_id

{'GPR155': 16448,
 'AP5Z1': 16449,
 'ELOB': 16450,
 'OSCP1': 16451,
 'USP12': 16453,
 'MAN1C1': 16454,
 'ZNF148': 16452,
 'POM121': 16455,
 'IL31RA': 16456,
 'EHD3': 16457,
 'C10orf120': 10,
 'AFF1': 16458,
 'CLDN11': 16459,
 'RFPL2': 16460,
 'LEFTY1': 16461,
 'NOL12': 16462,
 'ILDR1': 16464,
 'LEPR': 16463,
 'HSDL2': 16465,
 'RAI1': 16466,
 'TEX29': 16467,
 'ANKRD12': 16468,
 'ENTHD1': 16469,
 'HTR2B': 16470,
 'EFEMP1': 16471,
 'RIC8B': 16472,
 'TMEM205': 26773,
 'MGARP': 16473,
 'FAIM': 16474,
 'SH3TC1': 16475,
 'TMEM14A': 16476,
 'JAK2': 16477,
 'AKR1B15': 16478,
 'ARMCX2': 16479,
 'NUP133': 16480,
 'ZNF131': 16481,
 'DCLRE1C': 16482,
 'KCNK5': 16483,
 'CKAP2L': 16484,
 'LHFPL2': 16485,
 'KCNV1': 16486,
 'PDZD7': 16487,
 'BPNT1': 16488,
 'ADAMTS14': 16489,
 'PRPS1L1': 16492,
 'PRR12': 16491,
 'UNC13D': 16493,
 'FAN1': 16494,
 'C11orf65': 48,
 'HOXA10': 16496,
 'IL1F10': 16497,
 'TESPA1': 16498,
 'ZNF584': 16499,
 'TLR2': 16500,
 'MYO1B': 16502,
 'PMAIP1': 16501,
 'TMC7': 16503,
 'ZF

In [282]:
g1_id = [ sym_2_id[x] for x in gene_interactions_df_clean.g1 ]

In [283]:
g2_id = [ sym_2_id[x] for x in gene_interactions_df_clean.g2 ]

In [284]:
set(g1_id + g2_id)

{32768,
 32769,
 32770,
 32771,
 32772,
 32773,
 32774,
 32775,
 32776,
 32777,
 32778,
 10,
 32780,
 32781,
 32782,
 32783,
 32784,
 32785,
 32786,
 32787,
 32788,
 32789,
 32790,
 32791,
 32792,
 32793,
 32794,
 32795,
 32796,
 32797,
 32798,
 32799,
 32800,
 32801,
 32802,
 32803,
 32804,
 32805,
 32806,
 32807,
 32808,
 32809,
 32810,
 32811,
 32812,
 32813,
 32814,
 32815,
 48,
 32817,
 32818,
 32819,
 32820,
 32821,
 32816,
 32823,
 32824,
 32825,
 32826,
 32827,
 32828,
 32822,
 32830,
 32831,
 32832,
 32833,
 32834,
 32835,
 32836,
 32837,
 32838,
 32839,
 32840,
 32841,
 32842,
 32843,
 32844,
 32845,
 32846,
 32847,
 32848,
 32849,
 32850,
 32851,
 32852,
 32853,
 32854,
 32855,
 32856,
 32857,
 32858,
 32859,
 32860,
 32861,
 32862,
 32863,
 32864,
 32865,
 32866,
 32867,
 32868,
 32869,
 32870,
 32871,
 32872,
 32873,
 32874,
 32875,
 32876,
 32877,
 32878,
 32879,
 32880,
 32881,
 32882,
 32883,
 32884,
 32885,
 32886,
 32887,
 32888,
 32889,
 32890,
 32891,
 32892,
 32893

In [285]:
gene_interactions_id_df = pd.concat([pd.Series(g1_id), pd.Series(g2_id)], axis=1)
gene_interactions_id_df.columns = ['g1', 'g2']

In [286]:
gene_interactions_id_df.to_csv('gene_interactions_id.csv')

In [263]:
set(gene_interactions_id_df.g1)

{0,
 1,
 2,
 3,
 4,
 5,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 25,
 26,
 27,
 28,
 29,
 30,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 134,
 135,
 136,
 137,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 192,
 193,
 194,
 196,
 197,
 198,
 1

In [238]:
pd.read_csv('gene_interactions_id.csv', index_col=0)

,g1,g2
0,0,5620
1,0,5621
2,0,237
3,0,5563
4,0,5533
...,...,...
1498197,18891,11655
1498198,18891,13452
1498199,18891,13651
1498200,18894,6745


In [337]:
its_ = pd.read_csv('/Users/jiamingxu/Library/CloudStorage/OneDrive-NortheasternUniversity/Fall 2022/STAT/github/XNProject/Dataset/gene_connections/kidney_gene_connections.csv', index_col = 0)

In [345]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(its_.values.tolist(),  window=5, min_count=1, workers=4)
word_vectors = model.wv

In [349]:
model.docvecs

AttributeError: 'Word2Vec' object has no attribute 'docvecs'

In [343]:
[ x for x in its_ ]


['g1', 'g2']

In [344]:
its_.values.tolist()

[['OXCT2', 'ANLN'],
 ['OXCT2', 'ATP5E'],
 ['OXCT2', 'ATPAF1'],
 ['OXCT2', 'BCKDHA'],
 ['OXCT2', 'BCKDK'],
 ['OXCT2', 'C1QBP'],
 ['OXCT2', 'CBWD1'],
 ['OXCT2', 'CCT6B'],
 ['OXCT2', 'CPS1'],
 ['OXCT2', 'DARS2'],
 ['OXCT2', 'DDX19B'],
 ['OXCT2', 'ECH1'],
 ['OXCT2', 'ECHDC1'],
 ['OXCT2', 'ECT2'],
 ['OXCT2', 'HKDC1'],
 ['OXCT2', 'HSPD1'],
 ['OXCT2', 'ICAM5'],
 ['OXCT2', 'KRAS'],
 ['OXCT2', 'MCCC1'],
 ['OXCT2', 'MTO1'],
 ['OXCT2', 'NDUFAF7'],
 ['OXCT2', 'NFS1'],
 ['OXCT2', 'NME4'],
 ['OXCT2', 'OXCT1'],
 ['OXCT2', 'PMPCB'],
 ['OXCT2', 'RHOU'],
 ['OXCT2', 'RIT1'],
 ['OXCT2', 'TARS2'],
 ['OXCT2', 'TMLHE'],
 ['HTR2A', 'ARF1'],
 ['HTR2A', 'ARF6'],
 ['HTR2A', 'CBL'],
 ['HTR2A', 'CAV1'],
 ['HTR2A', 'DLG1'],
 ['HTR2A', 'DLG4'],
 ['HTR2A', 'HTR2A'],
 ['HTR2A', 'JAK2'],
 ['HTR2A', 'LYN'],
 ['HTR2A', 'UBC'],
 ['ADAMTS4', 'ADAM21'],
 ['ADAMTS4', 'ADAMTS2'],
 ['ADAMTS4', 'ATP2B2'],
 ['ADAMTS4', 'BIRC2'],
 ['ADAMTS4', 'BIRC6'],
 ['ADAMTS4', 'BMP8B'],
 ['ADAMTS4', 'CBWD2'],
 ['ADAMTS4', 'CBWD3'],
 ['ADAMTS

In [7]:
os.chdir('/Users/jiamingxu/Library/CloudStorage/OneDrive-NortheasternUniversity/Fall 2022/STAT/github/XNProject/Dataset/gene_interactions')

In [10]:
def get_files(path = './', ext = 'csv', string = 'gene'):
    """
    This function returns a list of files with specified extension and strings.
    """
    import os
    results=[]
    for file_ in os.scandir(path):
        if file_.is_file() and file_.name.endswith(ext) and string in file_.name:
            results.append(file_.name)
            
    return results

cancer_types = ['kidney', 'liver', 'pancreatic', 'bladder', 'ovarian', 'colon', 'breast', 'lung', 'leukemia']


In [35]:
dff = pd.DataFrame()
ne = 0
for file_ in get_files():
    df = pd.read_csv(file_, index_col=0)
    df = df[df.g1.isin(human_genes)]
    df = df[df.g2.isin(human_genes)]
    dff = pd.concat((dff, df), axis=0)
    ne += df.shape[0]

In [37]:
dff.drop_duplicates().shape

(570342, 2)

In [41]:
np.unique(dff.g1 + dff.g2).shape

(570341,)

In [14]:
human_genes

array(['AGPAT1', 'AGPAT2', 'AGPAT3', ..., 'XKRY', 'XKRY2', 'ZNF788P'],
      dtype=object)